**Twitter Credentials**

In [1]:
class TwitterCredentials():
  
  def __init__(self):
    self.app_key=""
    self.app_secret="" 
    self.oauth_token="" 
    self.oauth_token_secret=""

**Importing Libraries**

In [2]:
from tweepy import API 
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy import Cursor
from tweepy.streaming import StreamListener
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
from textblob import TextBlob

In [ ]:
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        twitter_credentials = TwitterCredentials()
        auth = OAuthHandler(twitter_credentials.app_key, twitter_credentials.app_secret)
        auth.set_access_token(twitter_credentials.oauth_token, twitter_credentials.oauth_token_secret)
        return auth


class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client


class TweetAnalyzer():

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[self.clean_tweet(tweet.text) for tweet in tweets], columns=['tweets'])
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['lang'] = np.array([tweet.lang for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        return df

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1
    

if __name__ == '__main__':

    api = TwitterClient().get_twitter_client_api()
    user_name = "BarackObama"
    tweets = api.user_timeline(screen_name=user_name, count=10, exclude_replies=False, include_retweets=True)
    tweet_analyzer = TweetAnalyzer()
    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
    df.to_csv(user_name+'_tweets.csv')
    print(df.head(10))
    tweets_list = df['tweets']
    summarized_tweet = '. '.join(tweets_list)
    print(summarized_tweet)